# 01_import dataset

## 01-1_import data

In [1]:
import os
import json
import pandas as pd

#https://drive.google.com/drive/folders/18qV82fNY3IIWu3BRoGqm_LNgJzE8Akbr?usp=drive_link
#base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
base_dir= '/Users/andyw.p.chen/Documents/Project/datasets'
#base_dir=  "c:\Users\user\Downloads\datasets"

def load_json_to_df(filename: str) -> pd.DataFrame:
    file_path = os.path.join(base_dir, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 如果是 { "target": {id: value, ...} }
    if isinstance(data, dict) and len(data) == 1 and isinstance(next(iter(data.values())), dict):
        key, inner = next(iter(data.items()))
        return pd.DataFrame(list(inner.items()), columns=["id", key])

    # dict of scalar
    if isinstance(data, dict):
        return pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])

    # list of dict
    elif isinstance(data, list):
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported JSON structure in {filename}: {type(data)}")


def load_csv_to_df(filename: str) -> pd.DataFrame:
    """讀取 CSV 並轉為 DataFrame。"""
    return pd.read_csv(os.path.join(base_dir, filename))

# JSON 資料
##mcc_codes_df = load_json_to_df("mcc_codes.json")
train_fraud_labels_df = load_json_to_df("train_fraud_labels.json")

# CSV 資料
cards_df = load_csv_to_df("cards_data.csv")
transactions_df = load_csv_to_df("transactions_data.csv")
users_df = load_csv_to_df("users_data.csv")

# 簡單檢查
#print(mcc_codes_df.head())
#print(train_fraud_labels_df.head())
#print(cards_df.head())
#print(transactions_df.head())
#print(users_df.apthead())


## 01-2_rename variable in each data set

In [2]:
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'id': 'transactions_id'})
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'target': 'is_fraud'})

cards_df = cards_df.rename(columns={'id':'card_id'})

users_df = users_df.rename(columns={'id':'client_id'})

transactions_df = transactions_df.rename(columns={'mcc': 'mcc_code'})
transactions_df = transactions_df.rename(columns={'id': 'transaction_id'})




## 01-3_變數型態統一及缺失值處理

In [3]:
def add_missing_flags(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    在 DataFrame 中對指定欄位建立 missing flag 欄位
    flag=1 表示缺失值，flag=0 表示非缺失值
    
    參數
    ----
    df : pd.DataFrame
        輸入的資料框
    cols : list
        要檢查的欄位名稱清單
    
    回傳
    ----
    pd.DataFrame : 新的資料框 (含新增的 flag 欄位)
    """
    for col in cols:
        df[f"{col}_missing_flag"] = df[col].isna().astype(int)
    return df

transactions_df = add_missing_flags(transactions_df, ["merchant_state", "zip", "errors"])

In [4]:
##train_fraud_labels_df##
train_fraud_labels_df["is_fraud"]=train_fraud_labels_df["is_fraud"].astype("category") 
train_fraud_labels_df["transactions_id"]=train_fraud_labels_df["transactions_id"].astype(int) #合併資料需要

##cards_df##
cards_df["card_brand"]=cards_df["card_brand"].astype("category") 
cards_df["card_type"]=cards_df["card_type"].astype("category")
#####不要load這行 cards_df["expires"]=pd.to_datetime(cards_df["expires"], format="%m/%Y")
cards_df["expires"] = pd.to_datetime(cards_df["expires"], format="%m/%Y").dt.to_period("M")
cards_df["has_chip"]=cards_df["has_chip"].astype("category")

cards_df['credit_limit'] = cards_df['credit_limit'].replace(r'[\$,]', '', regex=True).astype(int)
#####不要load這行 cards_df["acct_open_date"]=pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y")
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y").dt.to_period("M")
#####不要load這行 cards_df["year_pin_last_changed"]=pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y")
cards_df["year_pin_last_changed"] = pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y").dt.to_period("Y")
cards_df["card_on_dark_web"]=cards_df["card_on_dark_web"].astype("category") 

##users_df##
users_df["birth_year"] = pd.to_datetime(users_df["birth_year"], format="%Y").dt.to_period("Y")
users_df["birth_month"] = pd.to_datetime(users_df["birth_month"], format="%m").dt.to_period("M")
users_df["gender"]=users_df["gender"].astype("category") 
users_df['per_capita_income'] = users_df['per_capita_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['yearly_income'] = users_df['yearly_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['total_debt'] = users_df['total_debt'].replace(r'[\$,]', '', regex=True).astype(int)

##transactions_df##
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
#浮點數轉整數原因確定？
transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
##負數取log調成1
#transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float)

transactions_df["use_chip"]=transactions_df["use_chip"].astype("category") 

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'merchant_state'
] = 'online'

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'zip'
] = 20000 #原本是-1
## 我沒有全部改，這樣完之後仍有89006筆Missing，剩下都是在國外
transactions_df['zip'] = transactions_df['zip'].fillna(10000) #原本是-999
transactions_df["zip"]=transactions_df["zip"].astype("int64")

transactions_df['errors'] = transactions_df['errors'].astype('category')
transactions_df['errors'] = transactions_df['errors'].cat.add_categories('No_error').fillna('No_error')



In [5]:
#cars one hot encoding
##統一類別變數轉dummy variable(要注意共線性問題，應刪掉其中之一)

#card_type 原始種類：Debit_57%, Credit_33%, Debit(Prepaid)_9%
#card_brand 原始種類：MasterCard_52%, Visa_38%, Amex_7%, Discovery_3%
#has_chip 原始種類：Yes_89%, No_11%
#card_on_dark_web 原始種類：No_0%
cols_to_encode = ['card_type', 'card_brand', 'has_chip']
cards_df[cols_to_encode] = cards_df[cols_to_encode].astype('category')
dummies_cards = pd.get_dummies(
    cards_df[cols_to_encode], 
    prefix=cols_to_encode, 
    dtype='uint8'
    )
cards_df = pd.concat([cards_df, dummies_cards], axis=1)

#use_chip 原始種類：Swiped_52%, Chipe_36%, Online_12%
dummies_use = pd.get_dummies(transactions_df['use_chip'], prefix='use_chip', dtype='uint8')
transactions_df = pd.concat([transactions_df, dummies_use], axis=1)

#gender 原始種類：Female_51%, Male_49%
dummies_gender = pd.get_dummies(users_df['gender'], prefix='gender', dtype='uint8')
users_df = pd.concat([users_df, dummies_gender], axis=1)


cards_df.drop(columns=["has_chip_NO","has_chip"], inplace=True)
transactions_df.drop(columns=["use_chip"], inplace=True)
users_df.drop(columns=["gender_Female"], inplace=True)

# 02_資料整併成一張dataframe

## 02-1_資料整併

In [6]:
#transactions_df.loc[transactions_df["transaction_id"] == 10649266] #transaction_id vs id

#原始資料筆數：13305915
### transactions_df+train_fraud_labels_df      left 會有4390952 missing values
merged = pd.merge(transactions_df, train_fraud_labels_df, left_on="transaction_id", right_on="transactions_id", how="outer")
### transactions_df train_fraud_labels_df(8914963) + users_df 對過去不會有missing values
merged = pd.merge(merged,users_df , left_on="client_id", right_on="client_id", how="left")
### transactions_df train_fraud_labels_df users_df + cards_df 對過去不會有missing values
merged = pd.merge(merged,cards_df , left_on="card_id", right_on="card_id", how="left")

#刪掉重複的columns
merged.drop(columns=["transactions_id"], inplace=True)
merged.drop(columns=["client_id_y"], inplace=True)

## 合併完之後最後處理is_fraud(原會有missing values問題)
merged["is_fraud"] = merged["is_fraud"].astype(str)
merged.loc[merged['is_fraud'].str.lower() == 'no','is_fraud'] = '0'
merged.loc[merged['is_fraud'].str.lower() == 'yes','is_fraud'] = '1'
merged["is_fraud"] = pd.to_numeric(merged["is_fraud"], errors="coerce").astype("Int64")

merged = add_missing_flags(merged, ["is_fraud"])

#merged.to_csv("merged.csv", index=False)

# 先刪除不需要的DataFrame以節省記憶體
del transactions_df, users_df, cards_df, train_fraud_labels_df, cols_to_encode, dummies_cards, dummies_use, dummies_gender

In [7]:
backup_merged = merged.copy()
#merged = backup_merged.copy()

# 04_RFM features engineering model

## 04-1_資料進行變數轉換以求模型配飾更佳表現

In [ ]:
##有出事再趕快回復原狀
merged = backup_merged.copy()

In [8]:
import numpy as np
import pandas as pd

# 確保日期是 datetime 並排序
merged['date'] = pd.to_datetime(merged['date'])
merged = merged.sort_values(by=['client_id_x', 'date']).reset_index(drop=True)

# --- RecencyInterval ---
merged['RecencyInterval'] = merged.groupby('client_id_x')['date'].diff().dt.total_seconds().fillna(0)/60

# --- TxnFrequency for multiple windows (向量化滑動窗口) ---
window_days = [7, 30, 60, 90]
for w in window_days:
    merged[f'TxnFrequency_{w}d'] = 0

def compute_freq_vectorized(dates, windows):
    """向量化計算每筆交易在每個 window 內的交易數"""
    n = len(dates)
    dates_int = dates.values.astype('datetime64[D]').astype(int)
    res = {w: np.zeros(n, dtype=int) for w in windows}
    for w in windows:
        left = 0
        counts = np.zeros(n, dtype=int)
        for right in range(n):
            while dates_int[right] - dates_int[left] > w:
                left += 1
            counts[right] = right - left + 1
        res[w] = counts
    return res

# 分組計算
for cid, g in merged.groupby('client_id_x', sort=False):
    freq_dict = compute_freq_vectorized(g['date'], window_days)
    for w in window_days:
        merged.loc[g.index, f'TxnFrequency_{w}d'] = freq_dict[w]

# --- AmtDelta ---
merged['prev_amount'] = merged.groupby('client_id_x')['amount'].shift(1)
merged['AmtDelta'] = merged['amount'] - merged['prev_amount']
merged['AmtDelta'] = merged['AmtDelta'].fillna(0)
merged.drop(columns='prev_amount', inplace=True)

In [9]:
# US region mapping
us_region_map = {
    'Northeast': ['NY','NJ','PA','MA','CT','RI','NH','VT','ME'],
    'Midwest': ['IL','OH','MI','IN','WI','MN','IA','MO','ND','SD','NE','KS'],
    'South': ['FL','GA','SC','NC','AL','MS','LA','TX','OK','TN','KY','VA','WV','AR','MD','DE','DC'],
    'West': ['CA','WA','OR','NV','AZ','NM','CO','UT','ID','MT','WY','AK','HI'],
}
continent_map = {
    'Europe': [ ... ],  # 原本 continent_map['Europe'] 可直接使用
    'Online': ['online','AA']
}

us_region_lookup = {state: region for region, states in us_region_map.items() for state in states}

# --- 向量化 location 特徵 ---
merged['merchant_online'] = merged['merchant_state'].eq('online').astype('uint8')
merged['merchant_us'] = merged['merchant_state'].isin(us_region_lookup.keys()).astype('uint8')
merged['merchant_eu'] = merged['merchant_state'].isin(continent_map['Europe']).astype('uint8')
merged['merchant_others'] = (~merged[['merchant_online','merchant_us','merchant_eu']].any(axis=1)).astype('uint8')


In [10]:
# --- 首次交易標記 ---
merged['FirstTxnInRegion'] = (~merged.duplicated(subset=['client_id_x', 'merchant_state'])).astype('uint8')

In [11]:
# DifferentState
merged['prev_state']=(merged
                     .groupby('client_id_x')['merchant_state']
                     .shift(1))

merged['DifferentState'] = (
    (merged['merchant_state'] != merged['prev_state'])
    & merged['prev_state'].notna()
).astype(int)

merged = merged.drop(columns=['prev_state'])


In [12]:
#merged[["card_id","card_number"]]
import numpy as np
from scipy import stats 

# === (1) log轉換 ===
merged['amount'] = np.where(merged['amount'] < 0, 0, merged['amount'])  # 負數變 0
merged['amount'] = np.log(merged['amount'] + 1)  

# === (3) 平方根轉換 ===
merged['credit_limit']=np.sqrt(merged['credit_limit'])
merged['total_debt']=np.sqrt(merged['total_debt'])

# === (3) 立方根轉換 ===
merged['yearly_income']=np.cbrt(merged['yearly_income'])
merged['per_capita_income']=np.cbrt(merged['per_capita_income'])

## Box-Cox Transformation
###merged['yearly_income'], fitted_lambda = stats.boxcox(merged['yearly_income'])

# === (5) Yeo–Johnson 轉換（可處理負值） ===
###merged['per_capita_income'], lambdaValue =stats.yeojohnson(merged['per_capita_income'])

## 04-2_分割訓練集及測試集

In [13]:
# --- 選取數值型變數 ---
num_cols = merged.select_dtypes(include=['int64', 'float64','uint8','datetime64[ns]']).columns
df2 = merged[num_cols]

# --- dropna ---
df_cleaned = df2.dropna()
del df2

# --- 避免共線性 ---
df_cleaned.drop(columns=["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"], inplace=True)

# --- 確保 date 欄位在 df_cleaned 中 ---
if 'date' not in df_cleaned.columns:
    df_cleaned['date'] = merged.loc[df_cleaned.index, 'date']

# --- 依時間排序 ---
df_sorted = df_cleaned.sort_values('date')
df_sorted['year'] = df_sorted['date'].dt.year

# 2010–2011 → block 0
# 2012–2013 → block 1
# ...
# 2018–2019 → block 4（如果你真的是 2010–2019 共 10 年，會有 5 個 block）
df_sorted['time_block'] = (df_sorted['year'] - 2010) // 2


#對每個時間block做切割
train_list = []
test_list = []

for block_id, block_df in df_sorted.groupby('time_block'):
    print(f"\nProcessing Block {block_id}")

    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)

    train_block = block_df.iloc[:split_index].copy()
    test_block  = block_df.iloc[split_index:].copy()

    # -----------------------------
    # 1️⃣ 用「該 block 的 train」算 fraud rate
    # -----------------------------
    fraud_rate = (
        train_block
        .groupby('mcc_code')['is_fraud']
        .mean()
    )

    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index

    # -----------------------------
    # 2️⃣ 套用到該 block 的 train / test
    # -----------------------------
    train_block['HighRiskMCC'] = train_block['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_block['HighRiskMCC']  = test_block['mcc_code'].isin(high_risk_mcc).astype('uint8')

    train_list.append(train_block)
    test_list.append(test_block)


train_df = pd.concat(train_list).drop(columns=['date', 'year', 'time_block'])
test_df  = pd.concat(test_list).drop(columns=['date', 'year', 'time_block'])


#比例檢查
for block_id in sorted(df_sorted['time_block'].unique()):
    print(f"\nBlock {block_id}")
    print("Train fraud count:")
    print(train_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())
    print("Test fraud count:")
    print(test_df.loc[df_sorted['time_block'] == block_id, 'is_fraud'].value_counts())





Processing Block 0

Processing Block 1

Processing Block 2

Processing Block 3

Processing Block 4

Block 0
Train fraud count:
is_fraud
0    1353355
1       2610
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    338992
Name: count, dtype: Int64

Block 1
Train fraud count:
is_fraud
0    1433028
1       1152
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    357437
1      1108
Name: count, dtype: Int64

Block 2
Train fraud count:
is_fraud
0    1474582
1       1655
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    367862
1      1198
Name: count, dtype: Int64

Block 3
Train fraud count:
is_fraud
0    1493588
1       2448
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    373838
1       172
Name: count, dtype: Int64

Block 4
Train fraud count:
is_fraud
0    1367157
1       2393
Name: count, dtype: Int64
Test fraud count:
is_fraud
0    341792
1       596
Name: count, dtype: Int64


In [19]:
'''
result = pd.DataFrame(columns=[
    "Model", "Features", 
    "Train AUC", "Test AUC", 
    "Train PR AUC", "Test PR AUC"
])

'''

# ALL features

all_cols = ['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc_code',
       'errors', 'merchant_state_missing_flag', 'zip_missing_flag',
       'errors_missing_flag', 'use_chip_Chip Transaction',
       'use_chip_Online Transaction', 'use_chip_Swipe Transaction',
       'current_age', 'retirement_age', 'birth_year', 'birth_month', 'gender',
       'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards',
       'gender_Male', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web', 'card_type_Credit',
       'card_type_Debit', 'card_type_Debit (Prepaid)', 'card_brand_Amex',
       'card_brand_Discover', 'card_brand_Mastercard', 'card_brand_Visa',
       'has_chip_YES', 'is_fraud_missing_flag']

VIF_col = ["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"]

identifier = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

set_VIF = set(VIF_col)
set_identifier =set(identifier)
exclude_cols = set(VIF_col) | set(identifier)
all_cols = [x for x in all_cols if x not in exclude_cols]


# RFM features
rfm_cols = [
    'RecencyInterval', 'TxnFrequency_7d','TxnFrequency_30d',
    'TxnFrequency_60d', 'TxnFrequency_90d','AmtDelta'
]

# DK features
dk_cols = [
    'merchant_online', 'merchant_us', 'merchant_eu', 'merchant_others',
    'FirstTxnInRegion','HighRiskMCC','DifferentState'
]

# Grouping
feature_groups = {
    "X_all": all_cols,
    "X_rfm": rfm_cols,
    "X_dk": dk_cols,
    "X_all + X_rfm": all_cols + rfm_cols,
    "X_all + X_dk": all_cols + dk_cols,
    "X_rfm + X_dk": rfm_cols + dk_cols,
    "X_all + X_rfm + X_dk": all_cols + rfm_cols + dk_cols
}

## 04-3 老師建議我們先省略 Assumption: Avoid Multicollinearity

In [13]:
##處理共線性
train_df.drop(columns=["per_capita_income"], inplace=True)
train_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
train_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)
#再重跑一次VIF

In [14]:
test_df.drop(columns=["per_capita_income"], inplace=True)
test_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
test_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)

## 04-stepwise selection

In [21]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix


# ===========================================================
# Stepwise Logistic Regression with K control & tables
# ===========================================================
def stepwise_logit_with_k_tables(
    train_df,
    test_df,
    dep_var="is_fraud",
    k=314657018,
    threshold_in=0.05,
    threshold_out=0.10,
    verbose=True
):

    y_train = train_df[dep_var]
    X_train = train_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]
    X_test = test_df.drop(columns=[dep_var])

    included = []
    step = 0
    full_mode = (k == 314657018)

    # ======================================================
    # 🔸 特殊情況：k = 0（Intercept only）
    # ======================================================
    if k == 0:
        X_const = sm.add_constant(np.ones(len(y_train)), has_constant="add")
        final_model = sm.Logit(y_train, X_const).fit(disp=False)

        ll_full = final_model.llf
        ll_null = ll_full

        overall_fit = pd.DataFrame({
            "Measure": ["-2 Log Likelihood (−2LL) value"],
            "Value": [round(-2 * ll_full, 3)],
            "Change_from_Base": [""],
            "Change_pvalue": [""]
        })

        coef_df = pd.DataFrame({
            "Independent Variable": ["const"],
            "B": final_model.params.values,
            "Std. Error": final_model.bse.values,
            "Wald": [np.nan],
            "df": [1],
            "Sig.": [""],
            "Exp(B)": np.exp(final_model.params.values)
        })

        not_in_eq_df = pd.DataFrame({
            "Independent Variable": X_train.columns,
            "Score Statistic (LRT)": [None] * len(X_train.columns),
            "Significance": [None] * len(X_train.columns)
        })

        if verbose:
            print("✅ Stepwise completed with 0 variables (Intercept only).")

        return overall_fit, coef_df, not_in_eq_df, final_model

    # ======================================================
    # 🔁 Regular Stepwise (Forward + Backward)
    # ======================================================
    while True:
        step += 1
        changed = False

        # ---------- Forward ----------
        excluded = list(set(X_train.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)

        for new_var in excluded:
            try:
                X_try = sm.add_constant(
                    X_train[included + [new_var]],
                    has_constant="add"
                )
                model = sm.Logit(y_train, X_try).fit(disp=False)
                new_pvals[new_var] = model.pvalues[new_var]
            except Exception:
                new_pvals[new_var] = np.nan

        if new_pvals.empty:
            break

        best_pval = new_pvals.min()
        if best_pval < threshold_in:
            best_var = new_pvals.idxmin()
            included.append(best_var)
            changed = True
            if verbose:
                print(f"🟢 Step {step}: Forward add {best_var} (p={best_pval:.4g})")

        # ---------- Backward ----------
        if included:
            X_curr = sm.add_constant(X_train[included], has_constant="add")
            model = sm.Logit(y_train, X_curr).fit(disp=False)
            pvalues = model.pvalues.drop("const", errors="ignore")

            if not pvalues.empty:
                worst_pval = pvalues.max()
                if worst_pval > threshold_out:
                    worst_var = pvalues.idxmax()
                    included.remove(worst_var)
                    changed = True
                    if verbose:
                        print(f"🔴 Step {step}: Backward remove {worst_var} (p={worst_pval:.4g})")

        if not changed:
            if verbose:
                print(f"⚪ Step {step}: No change — stop.")
            break

        if not full_mode and len(included) >= k:
            if verbose:
                print(f"🟡 Reached k={k}, stop.")
            break

    # ======================================================
    # 🔹 Final Model
    # ======================================================
    if len(included) == 0:
        X_final = sm.add_constant(np.ones(len(y_train)), has_constant="add")
    else:
        X_final = sm.add_constant(X_train[included], has_constant="add")

    try:
        final_model = sm.Logit(y_train, X_final).fit(disp=False)
    except ValueError as e:
        if verbose:
            print("⚠️ Final model failed due to separation. Falling back to intercept-only.")
        X_const = sm.add_constant(np.ones(len(y_train)), has_constant="add")
        final_model = sm.Logit(y_train, X_const).fit(disp=False)
        included = []   # 非常重要：同步清空


    ll_full = final_model.llf
    ll_null = sm.Logit(
        y_train,
        sm.add_constant(np.ones(len(y_train)), has_constant="add")
    ).fit(disp=False).llf

    # ======================================================
    # 1️⃣ Overall Model Fit
    # ======================================================
    ll_diff = -2 * (ll_null - ll_full)
    df_diff = len(final_model.params) - 1
    p_value = stats.chi2.sf(ll_diff, df_diff)

    overall_fit = pd.DataFrame({
        "Measure": [
            "-2 Log Likelihood (−2LL) value",
            "Cox and Snell R2",
            "Nagelkerke R2",
            "Pseudo R2 (McFadden)",
            "Hosmer-Lemeshow χ2"
        ],
        "Value": [
            round(-2 * ll_full, 3),
            round(1 - np.exp((2 / len(y_train)) * (ll_null - ll_full)), 3),
            round(
                (1 - np.exp((2 / len(y_train)) * (ll_null - ll_full))) /
                (1 - np.exp(2 * ll_null / len(y_train))),
                3
            ),
            round(1 - (ll_full / ll_null), 3),
            round(ll_diff, 3)
        ],
        "Change_from_Base": [round(ll_diff, 3), "", "", "", ""],
        "Change_pvalue": [round(p_value, 4), "", "", "", ""]
    })

    # ======================================================
    # 2️⃣ Variables in the Equation
    # ======================================================
    coef_df = pd.DataFrame({
        "Independent Variable": final_model.params.index,
        "B": final_model.params.values,
        "Std. Error": final_model.bse.values,
        "Wald": (final_model.params / final_model.bse) ** 2,
        "df": 1,
        "Sig.": final_model.pvalues.values,
        "Exp(B)": np.exp(final_model.params.values)
    }).reset_index(drop=True)

    # ======================================================
    # 3️⃣ Variables Not in Equation
    # ======================================================
    excluded_vars = [v for v in X_train.columns if v not in included]
    not_in_eq = []

    for var in excluded_vars:
        try:
            X_tmp = sm.add_constant(
                X_train[included + [var]],
                has_constant="add"
            )
            temp_model = sm.Logit(y_train, X_tmp).fit(disp=False)
            lr_stat = -2 * (final_model.llf - temp_model.llf)
            p_val = stats.chi2.sf(lr_stat, 1)
            not_in_eq.append({
                "Independent Variable": var,
                "Score Statistic (LRT)": round(lr_stat, 3),
                "Significance": round(p_val, 4)
            })
        except Exception:
            not_in_eq.append({
                "Independent Variable": var,
                "Score Statistic (LRT)": None,
                "Significance": None
            })

    not_in_eq_df = pd.DataFrame(not_in_eq)

    # ======================================================
    # 🔹 Prediction matrices (safe)
    # ======================================================
    if len(included) == 0:
        X_train_pred = sm.add_constant(np.ones(len(y_train)), has_constant="add")
        X_test_pred = sm.add_constant(np.ones(len(y_test)), has_constant="add")
    else:
        X_train_pred = sm.add_constant(X_train[included], has_constant="add")
        X_test_pred = sm.add_constant(X_test[included], has_constant="add")

    train_probs = final_model.predict(X_train_pred)
    test_probs = final_model.predict(X_test_pred)

    train_pred = (train_probs > 0.5).astype(int)
    test_pred = (test_probs > 0.5).astype(int)

    train_acc = (train_pred == y_train).mean()
    test_acc = (test_pred == y_test).mean()

    train_auc = roc_auc_score(y_train, train_probs)
    test_auc = roc_auc_score(y_test, test_probs)
    train_pr_auc = average_precision_score(y_train, train_probs)
    test_pr_auc = average_precision_score(y_test, test_probs)

    if verbose:
        print(f"\n✅ Stepwise completed with {len(included)} variables")
        print(f"Train Accuracy: {train_acc:.4f}")
        print(f"Test Accuracy:  {test_acc:.4f}")
        print(f"Train AUC:     {train_auc:.4f}")
        print(f"Test AUC:      {test_auc:.4f}")
        print(f"Train PR-AUC:  {train_pr_auc:.4f}")
        print(f"Test PR-AUC:   {test_pr_auc:.4f}")

    return overall_fit, coef_df, not_in_eq_df, final_model


# ===========================================================
# Classification Table
# ===========================================================
def classification_table(model, df, target_col="is_fraud", cutoff=0.0015):

    y_true = df[target_col].astype(int)
    features = model.params.index.drop("const", errors="ignore")

    if len(features) == 0:
        X = sm.add_constant(np.ones(len(df)), has_constant="add")
    else:
        X = sm.add_constant(df[features], has_constant="add")

    y_pred_prob = model.predict(X)
    y_pred = (y_pred_prob >= cutoff).astype(int)

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    TP, FN, FP, TN = cm.ravel()

    fraud_total = TP + FN
    normal_total = FP + TN

    fraud_correct = TP / fraud_total if fraud_total else 0
    normal_correct = TN / normal_total if normal_total else 0
    overall_correct = (TP + TN) / (fraud_total + normal_total)

    return pd.DataFrame({
        "Actual Group": ["Fraud (1)", "Normal (0)", "Total"],
        "Predicted Fraud (1)": [TP, FP, TP + FP],
        "Predicted Normal (0)": [FN, TN, FN + TN],
        "Total": [fraud_total, normal_total, fraud_total + normal_total],
        "% Correct": [
            round(fraud_correct * 100, 1),
            round(normal_correct * 100, 1),
            round(overall_correct * 100, 1)
        ],
        "cutoff": [cutoff, "", ""],
        "F1 Score": [round(2 * TP / (2 * TP + FP + FN), 4), "", ""]
    })


# ===========================================================
# Feature Selector
# ===========================================================
def select_features(df, feature_list, dep_var="is_fraud"):

    cols = [c for c in feature_list if c in df.columns]
    df_sub = df[cols + [dep_var]]

    df_sub = df_sub.select_dtypes(include=[np.number])
    df_sub = df_sub.astype(float)

    return df_sub


In [ ]:
# 建立一個存放結果的字典
results = {}

# 為了避免修改到原始 feature_groups，我們先複製一份
import copy
current_feature_groups = copy.deepcopy(feature_groups)

for block_id, block_df in df_sorted.groupby('time_block'):

    print(f"\n{'='*40}")
    print(f"🚀 Running Block {block_id} (Year: {2010 + block_id*2} - {2011 + block_id*2})")
    print(f"{'='*40}")

    # 1. 確保時間排序並進行 80/20 切分
    block_df = block_df.sort_values('date')
    split_index = int(len(block_df) * 0.8)
    
    # 這裡先保留所有欄位，因為要算 HighRiskMCC
    train_raw = block_df.iloc[:split_index].copy()
    test_raw  = block_df.iloc[split_index:].copy()

    # -----------------------------
    # 2. ⚠️ 關鍵修正：在該 Block 的 Train 上計算 HighRiskMCC
    # -----------------------------
    # 計算 Fraud Rate
    fraud_rate = train_raw.groupby('mcc_code')['is_fraud'].mean()
    # 找出高風險 MCC (大於 2%)
    high_risk_mcc = fraud_rate[fraud_rate > 0.02].index

    # 映射回 Train 和 Test
    train_raw['HighRiskMCC'] = train_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    test_raw['HighRiskMCC']  = test_raw['mcc_code'].isin(high_risk_mcc).astype('uint8')
    # -----------------------------

    # 初始化該 Block 的結果存放區
    results[block_id] = {}

    # 3. 移除不參與訓練的非特徵欄位 (Date/Year/Block 等)
    # 這裡不刪除 transaction_id 等，因為後面 select_features 會挑選需要的
    drop_meta_cols = ['date', 'year', 'time_block']
    train_raw = train_raw.drop(columns=drop_meta_cols, errors='ignore')
    test_raw  = test_raw.drop(columns=drop_meta_cols, errors='ignore')

    # 🔁 對每一組 feature group 跑 stepwise
    for group_name, feature_list in current_feature_groups.items():

        print(f"\n   🔹 Processing Group: {group_name}")

        # 4. 特徵篩選 (使用你寫好的 select_features)
        # 確保 feature_list 裡的欄位都在 df 裡面 (防呆)
        valid_features = [f for f in feature_list if f in train_raw.columns]
        
        # 檢查是否有遺漏的欄位
        missing_features = set(feature_list) - set(valid_features)
        if missing_features:
            print(f"      ⚠️ Warning: {missing_features} not found in dataset. Skipping them.")

        if not valid_features:
            print("      ❌ No valid features found for this group. Skipping.")
            continue

        train_df = select_features(train_raw, valid_features, dep_var="is_fraud")
        test_df  = select_features(test_raw, valid_features, dep_var="is_fraud")
        
        # 5. 排除常數欄位 (Constant Columns) 以免 Logit 報錯
        # 若某個欄位在 train_df 裡只有一種數值 (例如全為 0)，Stepwise 會崩潰
        std_check = train_df.std()
        constant_cols = std_check[std_check == 0].index.tolist()
        if constant_cols:
            print(f"      ⚠️ Dropping constant columns: {constant_cols}")
            train_df = train_df.drop(columns=constant_cols)
            test_df = test_df.drop(columns=constant_cols)

        # 6. 執行 Stepwise
        try:
            overall_fit, coef_df, not_in_eq_df, final_model = stepwise_logit_with_k_tables(
                train_df,
                test_df,
                dep_var="is_fraud",
                k=314657018, # 設這麼大代表 Full Stepwise
                verbose=False # 建議 False，不然 output 會被洗版
            )

            # 7. 產生分類表 (Confusion Matrix info)
            train_table = classification_table(final_model, train_df)
            test_table  = classification_table(final_model, test_df)

            # 8. 儲存結果
            results[block_id][group_name] = {
                "overall_fit": overall_fit,
                "coef_df": coef_df,
                "not_in_eq_df": not_in_eq_df,
                "final_model": final_model,
                "train_table": train_table,
                "test_table": test_table
            }
            print(f"      ✅ Done. Final variables: {len(coef_df)}")

        except Exception as e:
            print(f"      ❌ Error processing {group_name} in Block {block_id}: {str(e)}")
            results[block_id][group_name] = {"error": str(e)}

print("\n🎉 All blocks and groups processed!")


🚀 Running Block 0 (Year: 2010 - 2011)

   🔹 Processing Group: X_all
      ⚠️ Warning: {'year_pin_last_changed', 'errors', 'card_type', 'card_on_dark_web', 'expires', 'merchant_city', 'address', 'date', 'birth_month', 'birth_year', 'card_brand', 'gender', 'merchant_state', 'acct_open_date'} not found in dataset. Skipping them.
      ⚠️ Dropping constant columns: ['use_chip_Chip Transaction']


/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/andyw.p.chen/Documents/Project/2025_Fraud-Detection-project/virtual/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return n

In [ ]:
## 分析用！！

results[
  block_id
][
  "X_all + X_rfm + X_dk"
]["coef_df"]


## 04-3 all丟模型內

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [16]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

Train AUC: 0.9154900057627916
Test AUC: 0.8854641260446391
Train PR-AUC: 0.08173481123106874
Test PR-AUC: 0.06417943887443732


In [17]:
cols_to_drop = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df  = test_df.drop(columns=cols_to_drop, errors='ignore')

In [ ]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])


NameError: name 'fit_full_logit' is not defined

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000,
        n_jobs=-1,
        class_weight="balanced"
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [16]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

Train AUC: 0.8963198198876533
Test AUC: 0.8912327090965326
Train PR-AUC: 0.04883650297518295
Test PR-AUC: 0.03905783518019472
